In [ ]:
# --- 1. IMPORTS, SETUPS E INICIALIZAÇÃO ---

import pandas as pd
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import numpy as np

print("--- 1. INICIANDO SETUP ---")

# Adiciona o diretório 'Code' ao path para encontrar o DataUtils
code_dir = Path('..').resolve() / 'Code'
if str(code_dir) not in sys.path:
    sys.path.append(str(code_dir))

import DataUtils as du
from DataUtils import AnalisadorBancario

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100) # Para ver mais nomes de bancos
plt.style.use('seaborn-v0_8-whitegrid')

output_dir = Path('..').resolve() / 'Output'
analisador = AnalisadorBancario(diretorio_output=str(output_dir))
print("-" * 50)

In [ ]:
# --- 2. TESTE DE CONSULTA BÁSICA (COSIF) ---
print("\n--- 2. TESTANDO CONSULTA BÁSICA COSIF ---")
# Use um nome exato da lista acima para garantir o sucesso
dados_bb_cosif = analisador.get_dados_cosif(
    identificador='BCO DO BRASIL S.A.',
    contas=['ATIVO REALIZÁVEL', 'Caixa'],
    datas=202403
)
print("Resultado para 'BCO DO BRASIL S.A.':")
display(dados_bb_cosif)
print("-" * 50)


# --- 3. TESTE DA TABELA COMPARATIVA COMPLETA ---
print("\n--- 3. TESTANDO TABELA COMPARATIVA COMPLETA ---")
indicadores_completos = {
    'Patrimônio Líquido': {'tipo': 'COSIF', 'conta': 'PATRIMÔNIO LÍQUIDO'},
    'Total Ativos': {'tipo': 'COSIF', 'conta': 'TOTAL GERAL DO ATIVO'},
    'Índice de Basileia': {'tipo': 'IFDATA', 'conta': 'Índice de Basileia'},
    'Situação': {'tipo': 'Atributo', 'atributo': 'SITUACAO_IFD_CAD'}
}

bancos_para_comparar = ['61190658', 'BANCO BRADESCO', 'NU FINANCEIRA', 'BANCO BTG PACTUAL', 'BCO DO BRASIL S.A.']

print("\nComparativo usando Documento 4060 (Balancete):")
tabela_balancete = analisador.comparar_indicadores(
    identificadores=bancos_para_comparar,
    indicadores=indicadores_completos,
    data=202403,
    documento_cosif=4060 
)
display(tabela_balancete)
print("-" * 50)


# --- 4. TESTE DA SÉRIE TEMPORAL ---
print("\n--- 4. TESTANDO SÉRIE TEMPORAL ---")
serie_nubank = analisador.get_serie_temporal_indicador(
    identificador='NU FINANCEIRA', # Usando nome curto
    conta='TOTAL GERAL DO ATIVO',
    data_inicio=202401,
    data_fim=202412,
    fonte='COSIF',
    documento_cosif=4060
)

print("\nSérie Temporal para 'TOTAL GERAL DO ATIVO' do Nubank:")
display(serie_nubank)

if not serie_nubank.empty:
    fig, ax = plt.subplots(figsize=(12, 6))
    serie_nubank.plot(ax=ax, marker='o', linestyle='-')
    ax.set_title("Evolução do Ativo Total - Nubank (COSIF Prudencial Doc 4060)", fontsize=16)
    ax.set_ylabel("Valor (em R$)")
    ax.set_xlabel("Data")
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    plt.tight_layout()
    plt.show()

print("\n--- FIM DOS TESTES ---")

In [ ]:
# ==============================================================================
# Gerar Tabela Comparativa a Partir de um Arquivo CSV (DEMORADO)
# ==============================================================================

# Garante que a pasta Results existe
results_dir = Path('.') / 'Results'
results_dir.mkdir(parents=True, exist_ok=True)

# Passo 1: Carregar a lista de CNPJs do seu arquivo CSV
print("--- Lendo a lista de bancos do arquivo Banks.csv ---")
try:
    caminho_csv = Path('.') / 'Banks.csv'
    
    df_bancos_interesse = pd.read_csv(caminho_csv, dtype={'CNPJ': str}, sep=';')
    
    # Agora que a coluna já é string, podemos simplesmente convertê-la para uma lista.
    # Não há mais necessidade de .astype(str) aqui.
    lista_de_bancos_do_csv = df_bancos_interesse['CNPJ'].tolist()
    
    # Verificação adicional: Padronizar cada item da lista para 8 dígitos
    # Isso garante que mesmo que o CSV tenha CNPJs de 14 dígitos, eles serão tratados corretamente.
    # Usaremos a função que já temos no DataUtils para isso!
    lista_de_bancos_do_csv = [du.standardize_cnpj_base8(cnpj) for cnpj in lista_de_bancos_do_csv]
    
    print(f"Encontrados {len(lista_de_bancos_do_csv)} bancos para análise.")
    print("Amostra da lista (padronizada para 8 dígitos):", lista_de_bancos_do_csv[:5])

except FileNotFoundError:
    print(f"ERRO: O arquivo 'Banks.csv' não foi encontrado no diretório 'Analysis/'.")
    lista_de_bancos_do_csv = []

# Só continua se a lista de bancos não estiver vazia
if lista_de_bancos_do_csv:
    # Passo 2: Definir os indicadores
    indicadores_desejados = {
        'Índice de Basileia': {'tipo': 'IFDATA', 'conta': 'Índice de Basileia'},
        'TCB': {'tipo': 'Atributo', 'atributo': 'TCB_IFD_CAD'},
        'Volume de Operações tipo G': {'tipo': 'IFDATA', 'conta': 'G'},
        'Volume de Operações tipo H': {'tipo': 'IFDATA', 'conta': 'H'},
        'Total de Volume de Operações': {'tipo': 'IFDATA', 'conta': 'Total'},
        'Depósitos a Prazo': {'tipo': 'IFDATA', 'conta': 'Depósitos a Prazo (a4)'},
        'Letra de Crédito Imobiliário': {'tipo': 'IFDATA', 'conta': 'Letras de Crédito Imobiliário (c1)'},
        'Letra de Crédito Agronegócio': {'tipo': 'IFDATA', 'conta': 'Letras de Crédito do Agronegócio (c2)'},
        'Letras Financeiras': {'tipo': 'IFDATA', 'conta': 'Letras Financeiras (c3)'},
        'RWA': {'tipo': 'IFDATA', 'conta': 'Patrimônio de Referência para Comparação com o RWA'},
        'Ativo Total': {'tipo': 'IFDATA', 'conta': 'Ativo Total'}
        }

    # Passo 3: Chamar a função 'comparar_indicadores'
    print("\n--- Gerando tabela comparativa para os bancos do CSV ---")
    data_analise = 202412

    tabela_final = analisador.comparar_indicadores(
        identificadores=lista_de_bancos_do_csv,
        indicadores=indicadores_desejados,
        data=data_analise,
        fillna=0
    )

    # Passo 4: Exibir e salvar o resultado
    display(tabela_final)

    try:
        # Salvar o relatório dentro de Results
        caminho_output_tabela = results_dir / f'relatorio_bancos_{data_analise}.xlsx'
        tabela_final.to_excel(caminho_output_tabela)
        print(f"\nRelatório salvo com sucesso em: '{caminho_output_tabela.resolve()}'")
    except Exception as e:
        print(f"\nErro ao salvar o relatório em Excel: {e}")

In [ ]:
# ==============================================================================
# CÉLULA DE TESTES DE ESTRESSE E CASOS DE BORDA
# ==============================================================================
print("--- INICIANDO TESTES DE ESTRESSE E CASOS DE BORDA ---")

# --- Teste 1: Grande Volume de Contas para um Único Banco ---
print("\n--- Teste 1: Puxando um grande volume de contas para o Banco do Brasil ---")
# Pegando as primeiras 50 contas do dicionário COSIF
contas_cosif_50 = analisador.df_cosif_prud['NOME_CONTA_COSIF'].dropna().unique()[:50].tolist()
data_teste = 202403

dados_volume = analisador.get_dados_cosif(
    identificador='BCO DO BRASIL S.A.',
    contas=contas_cosif_50,
    datas=data_teste,
    documentos=4060
)
print(f"Resultado: {len(dados_volume)} contas encontradas de 50 solicitadas.")
display(dados_volume.head())
print("-" * 50)


# --- Teste 2: Misturando Tipos de Identificadores na Tabela Comparativa ---
print("\n--- Teste 2: Tabela comparativa com múltiplos tipos de identificadores ---")
# Vamos usar nome curto, nome completo, CNPJ8 da entidade e CNPJ8 do líder
identificadores_mistos = [
    'ITAÚ',                                  # Nome curto
    'BANCO BRADESCO S.A.',                   # Nome completo
    '30680829',                              # CNPJ8 da Nu Financeira (entidade de interesse)
    '18236120'                               # CNPJ8 do Nu Pagamentos (líder que reporta)
]
indicadores_teste_2 = {
    'Patrimônio Líquido': {'tipo': 'COSIF', 'conta': 'PATRIMÔNIO LÍQUIDO'},
    'Situação': {'tipo': 'Atributo', 'atributo': 'SITUACAO_IFD_CAD'}
}

tabela_mista = analisador.comparar_indicadores(
    identificadores=identificadores_mistos,
    indicadores=indicadores_teste_2,
    data=data_teste
)
print("Resultado da tabela com identificadores mistos:")
display(tabela_mista)
print("-" * 50)


# --- Teste 3: Tratamento de Inputs Inválidos ---
print("\n--- Teste 3: Tratamento de identificadores e contas que não existem ---")
identificadores_invalidos = ['BANCO INEXISTENTE SA', '99999999']
contas_invalidas = {'Indicador Inexistente': {'tipo': 'COSIF', 'conta': 'CONTA QUE NAO EXISTE'}}

tabela_invalida = analisador.comparar_indicadores(
    identificadores=identificadores_invalidos,
    indicadores=contas_invalidas,
    data=data_teste
)
print("Resultado da tabela com inputs inválidos (esperado: tabela com erros e Nones):")
display(tabela_invalida)
print("-" * 50)


# --- Teste 4: Consulta COSIF Individual vs. Prudencial para a mesma entidade ---
print("\n--- Teste 4: Comparando dados Individuais vs. Prudenciais ---")
# Vamos usar o Banco do Brasil, que sabemos que reporta em ambos
conta_unica = 'TOTAL GERAL DO ATIVO'

print(f"\nBuscando '{conta_unica}' para 'BCO DO BRASIL S.A.' (PRUDENCIAL):")
dado_prudencial = analisador.get_dados_cosif(
    identificador='BCO DO BRASIL S.A.',
    contas=[conta_unica],
    datas=data_teste,
    tipo='prudencial',
    documentos=4060
)
display(dado_prudencial)

print(f"\nBuscando '{conta_unica}' para 'BCO DO BRASIL S.A.' (INDIVIDUAL):")
dado_individual = analisador.get_dados_cosif(
    identificador='BCO DO BRASIL S.A.',
    contas=[conta_unica],
    datas=data_teste,
    tipo='individual',
    documentos=4010 # Documento correspondente do individual
)
display(dado_individual)
print("-" * 50)


# --- Teste 5: Série Temporal com Tratamento de Zeros e Nulos ---
print("\n--- Teste 5: Série temporal com diferentes tratamentos de fillna ---")
# Usando uma conta que pode ter zeros, como 'Letras de Crédito Imobiliário'
banco_serie = 'BANCO INTER'
conta_serie = 'Letras de Crédito Imobiliário (c1)'
data_inicio = 202401
data_fim = 202412

print(f"\nSérie para '{conta_serie}' do {banco_serie} (Padrão: com 0 e NaN)")
serie_padrao = analisador.get_serie_temporal_indicador(
    identificador=banco_serie, conta=conta_serie, data_inicio=data_inicio,
    data_fim=data_fim, fonte='IFDATA'
)
display(serie_padrao)

print(f"\nMesma série (preenchendo ausentes com 0)")
serie_fill_zero = analisador.get_serie_temporal_indicador(
    identificador=banco_serie, conta=conta_serie, data_inicio=data_inicio,
    data_fim=data_fim, fonte='IFDATA', fillna=0
)
display(serie_fill_zero)

print(f"\nMesma série (tratando 0 como ausente)")
serie_zeros_nan = analisador.get_serie_temporal_indicador(
    identificador=banco_serie, conta=conta_serie, data_inicio=data_inicio,
    data_fim=data_fim, fonte='IFDATA', fillna=np.nan
)
display(serie_zeros_nan)
print("-" * 50)


# --- Teste 6: Tentativa de buscar dados de fontes cruzadas (deve falhar graciosamente) ---
print("\n--- Teste 6: Buscando conta COSIF na fonte IFDATA (deve retornar vazio) ---")
# Este teste verifica se a separação das fontes está funcionando.
dados_cruzados = analisador.get_dados_ifdata(
    identificador='BCO DO BRASIL S.A.',
    contas=['ATIVO REALIZÁVEL'], # Esta é uma conta COSIF
    datas=data_teste
)
print(f"Resultado da busca cruzada: {len(dados_cruzados)} linhas encontradas (esperado: 0).")
display(dados_cruzados)
print("\n--- FIM DOS TESTES DE ESTRESSE ---")

In [ ]:
# ==============================================================================
# CÉLULA DE TESTE: Consulta com Lista Mista de Nomes e Códigos de Contas
# ==============================================================================
print("\n--- Testando consulta com lista mista de nomes e códigos ---")

# Vamos buscar o Ativo Total pelo NOME e o Patrimônio Líquido pelo CÓDIGO
contas_mistas = [
    'TOTAL GERAL DO ATIVO', # Busca por nome (string)
    60000002              # Busca por código (int) para 'PATRIMÔNIO LÍQUIDO'
]
banco_teste_misto = 'BANCO BRADESCO S.A.'
data_teste_misto = 202403

# Chamando a função com a lista mista
dados_mistos = analisador.get_dados_cosif(
    identificador=banco_teste_misto,
    contas=contas_mistas,
    datas=data_teste_misto,
    documentos=4060
)

# O output já contém as colunas de código e nome, tornando-o legível.
print(f"Resultado da busca mista para '{banco_teste_misto}':")
display(dados_mistos[['BANCO_CONSULTADO', 'CONTA_COSIF', 'NOME_CONTA_COSIF', 'VALOR_CONTA_COSIF']])
print("-" * 50)